# OpenStreetMap Data Case Study

### Map Area

Las Vegas, Nevada

This map is of Vegas which I crave to visit someday in my life.

### Problems Encountered in the Map:

After initially downloading a small sample size of the Las Vegas area and running it against a provisional data.py file, I noticed five main problems with the data:

1. Inconsistent Postal Codes
2. Useless info in Changeset
3. Splitting Timestamp
4. 'FIXME' tags
5. Name tags split

In [1]:
import xml.etree.cElementTree as ET
import pprint 
from collections import defaultdict
import re
import pandas as pd

In [2]:
filename='las-vegas_nevada.osm'
sample_file='sample_las-vegas.osm'

## Initially, counting tag names. 

In [3]:
def count_tags(filename):
    tags={}
    for event,t in ET.iterparse(filename):
        if t.tag in tags.keys():
            tags[t.tag]+=1
        else:
            tags[t.tag]=1
    return tags

In [4]:
count_tags(sample_file)

{'member': 2,
 'nd': 2815,
 'node': 2629,
 'osm': 1,
 'relation': 1,
 'tag': 1277,
 'way': 282}

## In this project we clean data associated with nodes and ways, leaving out relations. 

In [5]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
expected = []
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [6]:
def get_tags(elem):
    tags=[]
    for tag in elem.findall('tag'):
        if PROBLEMCHARS.match(tag.attrib['k']):
            continue
        else:
            elem_tags={}
            elem_tags['id']=elem.attrib['id']
            elem_tags['value']=tag.attrib['v']
            try:
                split_key=tag.attrib['k'].split(':',1)
                elem_tags['type']=split_key[0]
                elem_tags['key']=split_key[1]
            except:
                elem_tags['key']=tag.attrib['k']
                elem_tags['type']='regular'
            tags.append(elem_tags)
    return tags
    
def nodes(filename):
    with open (filename) as f:
        all_nodes = []
        node_tags =[]
        for event, elem in ET.iterparse(filename, events=("start",)):
            if elem.tag=='node':
                node={}
                for att,values in elem.items():
                    node[att]=values
                all_nodes.append(node)
                elem_tags=get_tags(elem)
                node_tags.extend(elem_tags)            
    return pd.DataFrame(all_nodes),pd.DataFrame(node_tags)

def ways(filename):
    with open(filename) as f:
        all_ways = []
        way_nodes = []
        way_tags = []
        for event, elem in ET.iterparse(filename, events=("start",)):
            if elem.tag == 'way':
                way_attribs={}
                for att,values in elem.items():
                    way_attribs[att]=values
                all_ways.append(way_attribs)
                for i,nodes in enumerate(elem.findall('nd')):
                    way_n={}
                    way_n['id']=elem.attrib['id']
                    way_n['node_id']=nodes.attrib['ref']
                    way_n['position']=i
                    way_nodes.append(way_n)
                tags=get_tags(elem)
                way_tags.extend(tags)
    return pd.DataFrame(all_ways),pd.DataFrame(way_nodes),pd.DataFrame(way_tags)
                
                    

In [7]:
all_nodes,node_tags=nodes(filename)

In [8]:
all_ways, way_nodes, way_tags=ways(filename)

## I wish to edit all my dataframes into a form such that I can export a similar looking table to SQL.

## 1. all_nodes

In [9]:
all_nodes.head()

,changeset,id,lat,lon,timestamp,uid,user,version
0,45401766,31551114,36.1662859,-115.149225,2017-01-23T14:55:12Z,25975,AmZaf,10
1,15770302,31551289,36.0685037,-115.2037262,2013-04-18T07:52:34Z,1282393,Reno Editor,6
2,7129896,31551290,36.0677428,-115.2105885,2011-01-30T03:51:34Z,398919,gMitchellD,2
3,7129896,31551291,36.0675724,-115.2143331,2011-01-30T03:51:34Z,398919,gMitchellD,3
4,7129896,31551345,36.0672624,-115.2149755,2011-01-30T03:51:34Z,398919,gMitchellD,3


## Ignoring Changeset
Changeset does not provide us with any relevant info that is worth looking into, thus I will drop the column altogether. 
## Converting timestamp
Dropping 'Timestamp' to two columns as 'Time' and 'Date'

In [10]:
def func_for_date(row):
    return row['timestamp'].split('T')[0]
def func_for_time(row):
    return row['timestamp'].split('T',1)[1].strip('Z')

In [11]:
def edit_basic(df):
    del df['changeset']
    df['date']=df.apply(func_for_date,axis=1)
    df['time']=df.apply(func_for_time,axis=1)
    del df['timestamp']
    return df

In [12]:
all_nodes=edit_basic(all_nodes)

## Users can have a different SQL table identified uniquely by uid. 

In [13]:
def get_users(df):
    """a=set()
    for i in range(len(df)):
        b=(df.iloc[i]['uid'],df.iloc[i]['user'])
        a.add(b)"""
    a=df[['user','uid']]
    a=a.drop_duplicates()
    return a

In [14]:
all_users=get_users(all_nodes)
#a=pd.DataFrame(list(all_users),columns=['uid','user'])
del(all_nodes['user'])

In [15]:
all_users.head()

,user,uid
0,AmZaf,25975
1,Reno Editor,1282393
2,gMitchellD,398919
6,NE2,207745
7,woodpeck_repair,145231


In [16]:
all_nodes.head()

,id,lat,lon,uid,version,date,time
0,31551114,36.1662859,-115.149225,25975,10,2017-01-23,14:55:12
1,31551289,36.0685037,-115.2037262,1282393,6,2013-04-18,07:52:34
2,31551290,36.0677428,-115.2105885,398919,2,2011-01-30,03:51:34
3,31551291,36.0675724,-115.2143331,398919,3,2011-01-30,03:51:34
4,31551345,36.0672624,-115.2149755,398919,3,2011-01-30,03:51:34


## 2. node_tags

In [17]:
node_tags.head()

,id,key,type,value
0,31551114,is_in,regular,Nevada
1,31551114,continent,is_in,North America
2,31551114,country,is_in,USA
3,31551114,name,regular,Las Vegas
4,31551114,ar,name,لاس فيغاس


### We, see that there are some rows with info regarding Las Vegas, the city as a whole. We make a different df for these values. 
It quite noticeable that the first three rows are inconsistent or aren't uniform. We sort that out as well.


In [18]:
node_tags.iloc[0].type='is_in'
node_tags.iloc[0].key='state'
city_info=node_tags.loc[node_tags.id=='31551114']
del city_info['id']

In [19]:
city_info.head()

,key,type,value
0,state,is_in,Nevada
1,continent,is_in,North America
2,country,is_in,USA
3,name,regular,Las Vegas
4,ar,name,لاس فيغاس


## Editing Postal Codes

a. We see that some postal codes are rather specific in nature. For these we just take in the first five numbers. 
b. Others start with NV for Nevada. 
c. We skip the rest.

In [20]:
node_tags.loc[(node_tags.key=='postcode') & (node_tags.value.str.len()!=5)].head()

,id,key,type,value
26604,993789695,postcode,addr,89119-1001
33080,1700469005,postcode,addr,89109-1907
33197,1721875458,postcode,addr,8929
48425,2540047619,postcode,addr,NV 89117
48547,2572969955,postcode,addr,NV 89123


In [21]:
def edit_postcode(p):
    if len(p)>5:
        if p[5]=='-':
            return p[:5]
        elif p[0]=='N':
            return p[3:]
    return p

In [22]:
def edit_pcodes(df):
    for i in range(len(df)):
        if df.iloc[i]['key']=='postcode':
            df.iloc[i]['value']=edit_postcode(df.iloc[i]['value'])
    return df

In [23]:
node_tags=edit_pcodes(node_tags)

## 3. all_ways

In [24]:
all_ways.head()

,changeset,id,timestamp,uid,user,version
0,10364623,4879375,2012-01-11T21:28:27Z,92286,Paul Johnson,8
1,11581490,7149377,2012-05-12T22:59:41Z,121241,zephyr,7
2,49171267,14278330,2017-06-01T17:05:13Z,1330847,TheDutchMan13,3
3,49171267,14278332,2017-06-01T17:05:13Z,1330847,TheDutchMan13,3
4,47352899,14278334,2017-04-01T12:01:54Z,360392,maxerickson,3


### Editing the above Dataframe same as I did with all_nodes.

In [25]:
all_ways=edit_basic(all_ways)
all_ways.head()

,id,uid,user,version,date,time
0,4879375,92286,Paul Johnson,8,2012-01-11,21:28:27
1,7149377,121241,zephyr,7,2012-05-12,22:59:41
2,14278330,1330847,TheDutchMan13,3,2017-06-01,17:05:13
3,14278332,1330847,TheDutchMan13,3,2017-06-01,17:05:13
4,14278334,360392,maxerickson,3,2017-04-01,12:01:54


In [26]:
all_users=pd.concat([all_users,get_users(all_ways)])
del(all_ways['user'])

In [27]:
all_users=all_users.drop_duplicates()

In [28]:
all_users.describe()

,user,uid
count,1053,1053
unique,1053,1052
top,understan,605155
freq,1,2


In [29]:
all_ways.head()

,id,uid,version,date,time
0,4879375,92286,8,2012-01-11,21:28:27
1,7149377,121241,7,2012-05-12,22:59:41
2,14278330,1330847,3,2017-06-01,17:05:13
3,14278332,1330847,3,2017-06-01,17:05:13
4,14278334,360392,3,2017-04-01,12:01:54


## 4. way_tags

In [30]:
way_tags.head()

,id,key,type,value
0,4879375,ref,regular,CR 215
1,4879375,name,regular,Bruce Woodbury Beltway
2,4879375,FIXME,regular,check lanes; are bikes allowed?
3,4879375,lanes,regular,2
4,4879375,oneway,regular,yes


In [31]:
#way_tags postcodes
way_tags=edit_pcodes(way_tags)

In [32]:
way_tags.loc[way_tags.key=='FIXME']

,id,key,type,value
2,4879375,FIXME,regular,check lanes; are bikes allowed?
466,14279106,FIXME,regular,are bikes allowed?
473,14279108,FIXME,regular,are bikes allowed?
480,14279112,FIXME,regular,are bikes allowed?
588,14279371,FIXME,regular,verfiy bicycle=yes
623,14279500,FIXME,regular,verfiy bicycle=yes
629,14279505,FIXME,regular,are bikes allowed?
634,14279509,FIXME,regular,verfiy bicycle=yes
668,14279670,FIXME,regular,reconfigured?
1084,14281502,FIXME,regular,are bikes allowed?


In the above example we see, for 'FIXME' tag, data has been corrected already. For places, where it hasn't been, we have insufficient data to fix it. Thus, we remove all the 'FIXME' tags, in order to get a cleaner dataset.

In [33]:
way_tags=way_tags[way_tags.key!='FIXME']

Further, the name is split and stored in:

['name_1','name_base','name_base_1','name_direction_prefix','name_full']

This is not required as the final name is anyway stored in 'name'. Thus, it is redundant data and removing the tags with above mentioned keys.

By doing this we reduce the data from 2708 rows to 892 rows.


In [34]:
#Edit way_tags to remove random values of names
way_omit_tags=pd.Series(['name_1','name_base','name_base_1','name_direction_prefix','name_full','name_ty','name_type_1'])
way_tags=way_tags[~way_tags.key.isin(way_omit_tags)]

We kept 'name_type' as it holds relevant data in terms of the type 'way'.

In [35]:
map_name={'Aly':'Alley',
 'Ave':'Avenue',
 'Ave:Rd':'Drive',
 'Blvd':'Boulevard',
 'Br':'Brook',
 'Cir':'Circle',
 'Ct':'Court',
 'Ctr':'Drive',
 'Cv':'Cove',
 'Dr':'Drive',
 'Dr.':'Drive',
 'Dr:Rd':'Drive',
 'Dr; Dr; Dr; Rd':'Divided Highway',
 'Dr;Dr;Dr;Rd':'Divided Highway',
 'Hwy':'Highway',
 'Ln':'Lane',
 'Ln; Pky':'Parkway',
 'Loop':'Loop',
 'Mal':'Mall',
 'Pass':'Pass',
 'Path':'Path',
 'Pkwy':'Parkway',
 'Pky':'Parkway',
 'Pl':'Place',
 'Plz':'Plaza',
 'Rd':'Road',
 'Rd:St':'Street',
 'Rd; Blvd':'Boluevard',
 'Rd; Dr':'Drive',
 'Rd; Dr; Rd; St':'Road',
 'Rd; Rd; Dr; Rd':'Road',
 'Rte':'Route',
 'Spur':'Spur',
 'Sq':'Square',
 'St':'Street',
 'St:Trl':"Trail",
 'Ter':'Terrace',
 'Trl':'Trail',
 'Way':'Way',
 'Way:Way; Rd; Way':'Way',
 'Way; Rd; Way':'Way',
 'Xing':'Xing'}

In [36]:
way_tags['value'] = way_tags.loc[way_tags.key=='name_type'].value.map(map_name)

## 5. way_nodes

In [37]:
way_nodes.head()

,id,node_id,position
0,4879375,302158005,0
1,4879375,1127286209,1
2,4879375,302157999,2
3,4879375,1127286264,3
4,4879375,302158219,4


### We add the number of way  'nodes' to 'way_nodes' table. 

In [38]:
way_node_count = way_nodes.groupby('id').count().sort_values('position', ascending=False).reset_index()[["id",'position']]
way_node_count.columns=['id','node_count']
def add_count(row):
    return row.node_count
all_ways['node_count']=way_node_count.apply(add_count,axis=1)
del(way_node_count)

In [39]:
all_ways.head()

,id,uid,version,date,time,node_count
0,4879375,92286,8,2012-01-11,21:28:27,670.0
1,7149377,121241,7,2012-05-12,22:59:41,653.0
2,14278330,1330847,3,2017-06-01,17:05:13,646.0
3,14278332,1330847,3,2017-06-01,17:05:13,643.0
4,14278334,360392,3,2017-04-01,12:01:54,643.0


## Finally, getting a data frame for users so as it can be further converted into an SQL table.

## The final csv files are as:

In [40]:
#nodes
all_nodes.to_csv('all_nodes.csv',index=False)
node_tags.to_csv('node_tags.csv',index=False)
#ways
all_ways.to_csv('all_ways.csv',index=False)
way_nodes.to_csv('way_nodes.csv',index=False)
way_tags.to_csv('way_tags.csv',index=False)
#additional
city_info.to_csv('city_info.csv',index=False)
all_users.to_csv('all_users.csv',index=False)